In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('/Users/michaeltang/Documents/MIDS/Case Competition/Download/2020_Competition_Training.csv', low_memory = False)
test_data = pd.read_csv('/Users/michaeltang/Documents/MIDS/Case Competition/Download/2020_Competition_Holdout.csv', low_memory = False)

In [222]:
len(data['rucc_category'].unique())

9

In [2]:
print('training data shape: ', data.shape)
print('testing data shape: ', test_data.shape)

training data shape:  (69572, 826)
testing data shape:  (17681, 825)


In [3]:
y_data = data['transportation_issues']
data.drop(labels = ['transportation_issues'], axis = 1, inplace = True)

In [4]:
col_names = data.columns.tolist()
print('numbe of total indicators: ', len(col_names))

ind_col_names = [name for name in col_names if 'ind' in name]
print('number of binary indicators: ', len(ind_col_names))

non_ind_col_names = [name for name in col_names if name not in ind_col_names]
print('number of non-binary indicators: ', len(non_ind_col_names))

numbe of total indicators:  825
number of binary indicators:  354
number of non-binary indicators:  471


### group features into different categories, check for missing values and odd values inside each categories for both training and testing set determine how imputation will be performed ###

In [5]:
# betos codes
betos_col = [name for name in col_names if name.startswith('betos')] # no missing value
# behavioral health categories 
bh_col = [name for name in col_names if name.startswith('bh_')] # no missing values
# charlson comorbidity index value
cci_col = [name for name in col_names if name.startswith('cci_')] # no missing values
# clinical classification software code
ccsp_col = [name for name in col_names if name.startswith('ccsp_')] # no missing values
# Centers for Medicare and Medicaid Services
cms_col = [name for name in col_names if name.startswith('cms_')] # contains missing values ~ 3750, cms_ra_factor_type_cd contains '*' - 7
# cms level 2 diagnosis categories
cmsd2_col = [name for name in col_names if name.startswith('cmsd2_')] # no missing values
# county code (without state part)
cnty_col = [name for name in col_names if name.startswith('cnty_cd')] # contains 'other' = 46773
# KBM consensus data
cons_col = [name for name in col_names if name.startswith('cons_')] # contains missing values ~ 19278, cons_cmys contains '*' - 15
# all credit columns
credit_ALL_col = [name for name in col_names if name.startswith('credit_')]
# credit balance
credit_bal_col = [name for name in col_names if name.startswith('credit_bal_')] # contains missing values ~ 230
# credit household
credit_hh_col = [name for name in col_names if name.startswith('credit_hh_')] # contains missing values ~ 230
# high credit for consumer finance accounts 
credit_highcrd_col = [name for name in col_names if name.startswith('credit_highcrd_')] # contains missing values ~ 230
# mortgage account age
credit_minmob_col = [name for name in col_names if name.startswith('credit_minmob_')] # contains missing values ~ 230
# credit number
credit_num_col = [name for name in col_names if name.startswith('credit_num_')] # contains missing values ~ 231
# percentage balance
credit_prcnt_col = [name for name in col_names if name.startswith('credit_prcnt_')] # contains missing values ~ 231
# final diabetes cormobidity severity index score
dcsi_col = [name for name in col_names if name.startswith('dcsi_score')] # no missing values
# member age
age_col = [name for name in col_names if name.startswith('est_age')] # no missing values
# functional comorbidity index value
fci_col = [name for name in col_names if name.startswith('fci_score')] # no missing values
# sum of weighted HCC categories
hcc_col = [name for name in col_names if name.startswith('hcc_weighted_')] # no missing values
# Healthcare Effectiveness Data and Information Set
hedis_col = [name for name in col_names if name.startswith('hedis_')] # contains missing values ~ 70000, 54930, 1093
# health program - silver sneaker senior exercise program
hlth_col = [name for name in col_names if name.startswith('hlth_')] # no missing values
# abnormal lab results
lab_col = [name for name in col_names if name.startswith('lab_')] # no missing values
# language
lang_col = [name for name in col_names if name.startswith('lang_spoken_')] # contains 'E' = 1984
# medicare segmentation
mabh_col = [name for name in col_names if name.startswith('mabh_seg')] # contains 'UNK' = 12742
# per member per month by utilization category
med_col = [name for name in col_names if name.startswith('med_')] # no missing values
# percent days covered for each category of drugs 
pdc_col = [name for name in col_names if name.startswith('pdc_')] # contains '1.1'
# physician E&M category
phy_col = [name for name in col_names if name.startswith('phy_')] # no missing values
# prov_spec_cd
prov_col = [name for name in col_names if name.startswith('prov_')] # no missing values
# revenue code CMS categories
rev_col = [name for name in col_names if name.startswith('rev_')] # no missing values
# rural urban continuum code
rucc_col = [name for name in col_names if name.startswith('rucc_category')] # no missing values
# per member per month count for prescription
rx_col = [name for name in col_names if name.startswith('rx_')] # no missing values
# standard humana member identifier
sdr_col = [name for name in col_names if name.startswith('person_id_')] # no missing values
# member gender
sex_col = [name for name in col_names if name.startswith('sex_cd')] # no missing values
# smoker based on the presence of smoking indication
smoker_col = [name for name in col_names if name.startswith('smoker_')] # no missing values
# humana member identifier
src_col = [name for name in col_names if name.startswith('src_platform')] # no missing values
# geographic information - postal abbreviation
state_col = [name for name in col_names if name.startswith('state_cd')] # contains 'other' = 46773
# per member per month count of logical cliams for each of the MCC categories
submcc_col = [name for name in col_names if name.startswith('submcc_')] # no missing values
# combined Med & B values by utilization category
total_col = [name for name in col_names if name.startswith('total_')] # no missing values
# zip code
zip_col = [name for name in col_names if name.startswith('zip_cd')] # contains 'other' = 46773

In [7]:
master_data = data.copy()
master_test_data = test_data.copy()

In conclusion:
1. new zip_col with 2 digits
2. replace '*', 'other', 'UNK',  with null
3. convert 'cons_cmys', 'zip_cd' to numeric
4. with rucc: new zip_col
5. with new zip: 'cons_n2mob', 'cons_n2pbl', 'cons_n2pmv', 'cons_n65p_y', 'cons_cmys', 'cons_hhcomp', 'cons_homstat', all credit cols, lang_col
6. with hcc_weighted_sum: cms_col
7. with cci_score: 'cons_hcaccprf_h', 'cons_hcaccprf_p', all mabh_col
8. with dcsi: all hedis_col
9. with est_age: 'cons_n2029_y', 'cons_n65p_y'
10. turn pdc_col in binary
11. drop column: 'cons_online_buyer', 'cons_ret_y', 'cons_retail_buyer', 'cons_veteran_y', 'hedis_ami', 'hedis_cmc_ldc_c_control', 'hedis_cmc_ldc_c_screen', 'cnty_col', 'state_col', 'person_id_syn', 'src_platform_cd'

In [8]:
# check for categorical variables
cat_cols = [name for name in col_names if master_data[name].dtypes == object]
print(cat_cols)

['person_id_syn', 'src_platform_cd', 'sex_cd', 'lang_spoken_cd', 'mabh_seg', 'cms_ra_factor_type_cd', 'cons_cmys', 'cons_hhcomp', 'cons_homstat', 'hedis_ami', 'hedis_cmc_ldc_c_control', 'hedis_cmc_ldc_c_screen', 'hedis_dia_eye', 'hedis_dia_hba1c_ge9', 'hedis_dia_hba1c_test', 'hedis_dia_ldc_c_control', 'hedis_dia_ldc_c_screen', 'hedis_dia_ma_nephr', 'rucc_category', 'zip_cd', 'cnty_cd', 'state_cd']


In [9]:
# keep the first 2 digit of a zip code
master_data['zip_cd'] = master_data['zip_cd'].apply(lambda x: x[:2] if x != 'other' else x)
master_test_data['zip_cd'] = master_test_data['zip_cd'].apply(lambda x: x[:2] if x != 'other' else x)
# keep the first letter of the medicare segmentation
master_data['mabh_seg'] = master_data['mabh_seg'].apply(lambda x: x[0] if x != 'UNK' else x)
master_test_data['mabh_seg'] = master_test_data['mabh_seg'].apply(lambda x: x[0] if x != 'UNK' else x)
# keep the first letter of the rucc code
master_data['rucc_category'] = master_data['rucc_category'].apply(lambda x: x[0] if x != 'UNK' else x)
master_test_data['rucc_category'] = master_test_data['rucc_category'].apply(lambda x: x[0] if x != 'UNK' else x)
# replace artifacts with nan
master_data.replace('*', np.nan, inplace = True)
master_data.replace('other', np.nan, inplace = True)
master_data.replace('UNK', np.nan, inplace = True)
master_test_data.replace('*', np.nan, inplace = True)
master_test_data.replace('other', np.nan, inplace = True)
master_test_data.replace('UNK', np.nan, inplace = True)
# replace unknown records with nan
master_data['cons_cmys'].replace('0', np.nan, inplace = True)
master_data['cons_hhcomp'].replace('U', np.nan, inplace = True)
master_data['cons_homstat'].replace('U', np.nan, inplace = True)
master_test_data['cons_cmys'].replace('0', np.nan, inplace = True)
master_test_data['cons_hhcomp'].replace('U', np.nan, inplace = True)
master_test_data['cons_homstat'].replace('U', np.nan, inplace = True)
# define a list for columns to be converted from object to float
string_to_num_cols = ['cons_cmys']
# convert columns to float
master_data[string_to_num_cols] = master_data[string_to_num_cols].apply(pd.to_numeric)
master_test_data[string_to_num_cols] = master_test_data[string_to_num_cols].apply(pd.to_numeric)

In [10]:
# produce mapping
rucc_mapping = master_data.groupby(['rucc_category'])[zip_col].apply(lambda x: x.value_counts().index[0]).to_dict()

zip_mapping_col = ['cons_n2mob', 'cons_n2pbl', 'cons_n2pmv', 'cons_n65p_y', 'cons_cmys', 
                   'cons_hhcomp', 'cons_homstat'] + credit_ALL_col + lang_col
zip_mapping = master_data.groupby(['zip_cd'])[zip_mapping_col].apply(lambda x: x.value_counts().index[0]).to_dict()

hcc_weighted_sum_mapping = master_data.groupby(['hcc_weighted_sum'])[cms_col].apply(lambda x: x.value_counts().index[0]).to_dict()

cci_score_mapping_col = ['cons_hcaccprf_h', 'cons_hcaccprf_p'] + mabh_col
cci_score_mapping = master_data.groupby(['cci_score'])[cci_score_mapping_col].apply(lambda x: x.value_counts().index[0]).to_dict()

dcsi_mapping = master_data.groupby(['dcsi_score'])[hedis_col].apply(lambda x: x.value_counts().index[0]).to_dict()

est_age_mapping_col = ['cons_n2029_y', 'cons_n65p_y']
est_age_mapping = master_data.groupby(['est_age'])[est_age_mapping_col].apply(lambda x: x.value_counts().index[0]).to_dict()

In [11]:
# define a function to perform mapping of missing values
def map_missing(num_rows, mapping, source_col, target_col, data):
    for i in range(num_rows):
        for j, col in enumerate(target_col):
            if pd.isna(data.loc[i, col]):
                data.loc[i, col] = mapping[data.loc[i, source_col]][j]
                pass
            pass
        pass

In [12]:
# define the height of each dataframe
master_data_row = master_data.shape[0]
master_test_data_row = master_test_data.shape[0]
# perform mapping
map_missing(master_data_row, rucc_mapping, 'rucc_category', zip_col, master_data)
map_missing(master_test_data_row, rucc_mapping, 'rucc_category', zip_col, master_test_data)

map_missing(master_data_row, zip_mapping, 'zip_cd', zip_mapping_col, master_data)
map_missing(master_test_data_row, zip_mapping, 'zip_cd', zip_mapping_col, master_test_data)

map_missing(master_data_row, hcc_weighted_sum_mapping, 'hcc_weighted_sum', cms_col, master_data)
map_missing(master_test_data_row, hcc_weighted_sum_mapping, 'hcc_weighted_sum', cms_col, master_test_data)

map_missing(master_data_row, cci_score_mapping, 'cci_score', cci_score_mapping_col, master_data)
map_missing(master_test_data_row, cci_score_mapping, 'cci_score', cci_score_mapping_col, master_test_data)

map_missing(master_data_row, dcsi_mapping, 'dcsi_score', hedis_col, master_data)
map_missing(master_test_data_row, dcsi_mapping, 'dcsi_score', hedis_col, master_test_data)

map_missing(master_data_row, est_age_mapping, 'est_age', est_age_mapping_col, master_data)
map_missing(master_test_data_row, est_age_mapping, 'est_age', est_age_mapping_col, master_test_data)

In [13]:
# define columns to be dropped
drop_col = ['cons_online_buyer', 'cons_ret_y', 'cons_retail_buyer', 'cons_veteran_y', 'hedis_ami', 'hedis_cmc_ldc_c_control', 
            'hedis_cmc_ldc_c_screen', 'cnty_cd', 'state_cd', 'person_id_syn', 'src_platform_cd']
# drop columns
master_data.drop(drop_col, axis = 1, inplace = True, errors = 'ignore')
master_test_data.drop(drop_col, axis = 1, inplace = True, errors = 'ignore')

In [14]:
# export imputed file to csv
master_data.to_csv('imputed_training.csv', index = False)
master_test_data.to_csv('imputed_holdout.csv', index = False)

In [15]:
# inputed_data = pd.read_csv('imputed_training.csv', low_memory = False)
# inputed_test_data = pd.read_csv('imputed_holdout.csv', low_memory = False)

imputed_data = master_data.copy()
imputed_test_data = master_test_data.copy()

In [16]:
# imputed_data['zip_cd'] = imputed_data['zip_cd'].astype('object')
# imputed_test_data['zip_cd'] = imputed_data['zip_cd'].astype('object')
#check for categorical variables
cat_cols = [name for name in imputed_data.columns if imputed_data[name].dtypes == object]
cat_cols

['sex_cd',
 'lang_spoken_cd',
 'mabh_seg',
 'cms_ra_factor_type_cd',
 'cons_hhcomp',
 'cons_homstat',
 'hedis_dia_eye',
 'hedis_dia_hba1c_ge9',
 'hedis_dia_hba1c_test',
 'hedis_dia_ldc_c_control',
 'hedis_dia_ldc_c_screen',
 'hedis_dia_ma_nephr',
 'rucc_category',
 'zip_cd']

In [17]:
ordinal_cat_cols = ['mabh_seg', 'cons_homstat', 'rucc_category']
nominal_cat_cols = ['sex_cd', 'lang_spoken_cd', 'cms_ra_factor_type_cd', 'cons_hhcomp', 'hedis_dia_eye', 'hedis_dia_hba1c_ge9',
                   'hedis_dia_hba1c_test', 'hedis_dia_ldc_c_control', 'hedis_dia_ldc_c_screen', 'hedis_dia_ma_nephr', 'zip_cd']
binary_cols = pdc_col

In [18]:
# encode pdc_col
imputed_data[pdc_col] = (imputed_data[pdc_col] != 1.1).astype('int32')
imputed_test_data[pdc_col] = (imputed_test_data[pdc_col] != 1.1).astype('int32')

In [19]:
# encode ordinal_cat_cols
ordinal_mapping = [{'col':'mabh_seg', 'mapping':{'H':1, 
                                                 'C':0}},
                   {'col':'cons_homstat', 'mapping':{'T':0,
                                                     'R':1,
                                                     'P':2,
                                                     'Y':3}},
                   {'col':'rucc_category', 'mapping':{'1':9, '2':8, '3':7,
                                                      '4':6, '5':5, '6':4,
                                                      '7':3, '8':2, '9':1}}]

import category_encoders as ce
ordinal_encoder = ce.OrdinalEncoder(ordinal_cat_cols, mapping = ordinal_mapping)
imputed_data = ordinal_encoder.fit_transform(imputed_data)
imputed_test_data = ordinal_encoder.transform(imputed_test_data)

In [20]:
ohe_encoder = ce.OneHotEncoder(cols = nominal_cat_cols, return_df = True, use_cat_names= True)
imputed_data = ohe_encoder.fit_transform(imputed_data)
imputed_test_data = ohe_encoder.transform(imputed_test_data)

/usr/local/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [21]:
# export imputed file to csv
imputed_data.to_csv('encoded_training.csv', index = False)
imputed_test_data.to_csv('encoded_holdout.csv', index = False)

In [235]:
# encoded_data = pd.read_csv('encoded_training.csv', low_memory = False)
# encoded_test_data = pd.read_csv('encoded_holdout.csv', low_memory = False)

encoded_data = imputed_data.copy()
encoded_test_data = imputed_test_data.copy()

In [236]:
encoded_cols = encoded_data.columns

In [237]:
betos_col = [name for name in encoded_cols if name.startswith('betos_')] # 0:pmpm_ct, 1:_ind
bh_col = [name for name in encoded_cols if name.startswith('bh_')] # 1:_ind
cci_col = [name for name in encoded_cols if name.startswith('cci_')] # 0:_score
ccsp_col = [name for name in encoded_cols if name.startswith('ccsp_')] # 1:_ind, ccsp_034_ind, ccsp_120_ind: all 0
cms_col = [name for name in encoded_cols if name.startswith('cms_')] # 0: _amt, _nbr, 1: _ind, cd_%%
cmsd2_col = [name for name in encoded_cols if name.startswith('cmsd2_')] # 1:_ind
cons_col = [name for name in encoded_cols if name.startswith('cons_')] # use all 
credit_ALL_col = [name for name in encoded_cols if name.startswith('credit_')] 
credit_bal_col = [name for name in encoded_cols if name.startswith('credit_bal_')] # all continuous
credit_hh_col = [name for name in encoded_cols if name.startswith('credit_hh_')] # all continuous
credit_highcrd_col = [name for name in encoded_cols if name.startswith('credit_highcrd_')] # all continuous
credit_minmob_col = [name for name in encoded_cols if name.startswith('credit_minmob_')] # all continuous
credit_num_col = [name for name in encoded_cols if name.startswith('credit_num_')] # all continuous
credit_prcnt_col = [name for name in encoded_cols if name.startswith('credit_prcnt_')] # all continuous
dcsi_col = [name for name in encoded_cols if name.startswith('dcsi_score')] # 0:dcsi_score
age_col = [name for name in encoded_cols if name.startswith('est_age')] # 0: est_age
fci_col = [name for name in encoded_cols if name.startswith('fci_score')] # 0: fci_score
hcc_col = [name for name in encoded_cols if name.startswith('hcc_weighted_')] # 0: hcc_weighted_sum
hedis_col = [name for name in encoded_cols if name.startswith('hedis_')] # all binary
hlth_col = [name for name in encoded_cols if name.startswith('hlth_')] # 1: _par_status, _pct_par..hlth_pgm_slvrsnkr_refer_status: all 0
lab_col = [name for name in encoded_cols if name.startswith('lab_')] # 1:_ind
lang_col = [name for name in encoded_cols if name.startswith('lang_spoken_')] # all binary
mabh_col = [name for name in encoded_cols if name.startswith('mabh_seg')] # all binary
med_col = [name for name in encoded_cols if name.startswith('med_')] # all continuous, 
# [med_ip_ltach_admit_ct_pmpm, med_ip_ltach_admit_days_pmpm, med_ip_maternity_admit_ct_pmpm, med_ip_maternity_admit_days_pmpm,
# med_ip_mhsa_admit_ct_pmpm, med_ip_mhsa_admit_days_pmpm] all 0
pdc_col = [name for name in encoded_cols if name.startswith('pdc_')] # all binary
phy_col = [name for name in encoded_cols if name.startswith('phy_')] # all binary, 0: _ind
prov_col = [name for name in encoded_cols if name.startswith('prov_')] # all binary, 0: _ind
rev_col = [name for name in encoded_cols if name.startswith('rev_')] # all binary, 0: _ind ... rev_cms_nicu_ind: all 0
rucc_col = [name for name in encoded_cols if name.startswith('rucc_category')] # continuous
rx_col = [name for name in encoded_cols if name.startswith('rx_')] # 0:pmpm_ct, 1:_ind ... 
# ['rx_gpi2_45_ind', 'rx_gpi2_84_ind', 'rx_gpi2_81_ind', 'rx_gpi2_29_ind',
#        'rx_gpi2_14_ind', 'rx_gpi2_19_ind', 'rx_gpi2_92_ind', 'rx_gpi2_95_ind',
#        'rx_gpi2_08_ind', 'rx_gpi2_69_ind', 'rx_gpi2_76_ind', 'rx_gpi2_80_ind',
#        'rx_gpi2_98_ind', 'rx_gpi2_20_ind', 'rx_gpi2_70_ind', 'rx_gpi2_96_ind']
sex_col = [name for name in encoded_cols if name.startswith('sex_cd')] # all binary
smoker_col = [name for name in encoded_cols if name.startswith('smoker_')] # all binary
submcc_col = [name for name in encoded_cols if name.startswith('submcc_')] # 0:pmpm_ct, 1:_ind ... 
# ['submcc_pre_del_ind', 'submcc_rar_drm_ind', 'submcc_ben_lymp_ind',
#        'submcc_pre_othr_ind', 'submcc_rar_othr_ind', 'submcc_hdz_surg_ind',
#        'submcc_cad_fh/ho_ind', 'submcc_neo_fh/ho_ind', 'submcc_rar_als_ind',
#        'submcc_rar_cf_ind', 'submcc_rar_sca_ind', 'submcc_inf_men_ind',
#        'submcc_hdz_it_i_ind', 'submcc_trm_fxu_ind', 'submcc_hiv_pcp_ind',
#        'submcc_rar_pol_ind', 'submcc_rsk_fh/h_ind', 'submcc_rsk_pcos_ind',
#        'submcc_pre_ect_ind', 'submcc_pre_mul_ind', 'submcc_brn_acc_ind',
#        'submcc_rsk_an_ind', 'submcc_hiv_kapo_ind', 'submcc_rsk_othr_ind']
total_col = [name for name in encoded_cols if name.startswith('total_')] # 0:pmpm_ct
# ['total_ip_ltach_admit_ct_pmpm', 'total_ip_ltach_admit_days_pmpm',
#        'total_ip_maternity_admit_ct_pmpm',
#        'total_ip_maternity_admit_days_pmpm']

zip_col = [name for name in encoded_cols if name.startswith('zip_cd')] # all start with zip_cd

In [238]:
# drop columns that contains all 0 values
drop_cols = ['ccsp_034_ind', 'ccsp_120_ind', 'hlth_pgm_slvrsnkr_refer_status', 'med_ip_ltach_admit_ct_pmpm', 
             'med_ip_ltach_admit_days_pmpm', 'med_ip_maternity_admit_ct_pmpm', 'med_ip_maternity_admit_days_pmpm', 
             'med_ip_mhsa_admit_ct_pmpm', 'med_ip_mhsa_admit_days_pmpm', 'rx_gpi2_45_ind', 'rx_gpi2_84_ind', 
             'rx_gpi2_81_ind', 'rx_gpi2_29_ind', 'rx_gpi2_14_ind', 'rx_gpi2_19_ind', 'rx_gpi2_92_ind', 
             'rx_gpi2_95_ind', 'rx_gpi2_08_ind', 'rx_gpi2_69_ind', 'rx_gpi2_76_ind', 'rx_gpi2_80_ind',
             'rx_gpi2_98_ind', 'rx_gpi2_20_ind', 'rx_gpi2_70_ind', 'rx_gpi2_96_ind', 'submcc_pre_del_ind', 
             'submcc_rar_drm_ind', 'submcc_ben_lymp_ind','submcc_pre_othr_ind', 'submcc_rar_othr_ind', 
             'submcc_hdz_surg_ind', 'submcc_cad_fh/ho_ind', 'submcc_neo_fh/ho_ind', 'submcc_rar_als_ind', 
             'submcc_rar_cf_ind', 'submcc_rar_sca_ind', 'submcc_inf_men_ind', 'submcc_hdz_it_i_ind', 
             'submcc_trm_fxu_ind', 'submcc_hiv_pcp_ind', 'submcc_rar_pol_ind', 'submcc_rsk_fh/h_ind', 
             'submcc_rsk_pcos_ind', 'submcc_pre_ect_ind', 'submcc_pre_mul_ind', 'submcc_brn_acc_ind',
             'submcc_rsk_an_ind', 'submcc_hiv_kapo_ind', 'submcc_rsk_othr_ind', 'total_ip_ltach_admit_ct_pmpm', 
             'total_ip_ltach_admit_days_pmpm', 'total_ip_maternity_admit_ct_pmpm', 'total_ip_maternity_admit_days_pmpm',
             'rev_cms_nicu_ind']
# drop columns
encoded_data.drop(drop_cols, axis = 1, inplace = True, errors = 'ignore')
encoded_test_data.drop(drop_cols, axis = 1, inplace = True, errors = 'ignore')

In [239]:
# drop redundant nonbinary information 
nonbin_betos = [n for n in betos_col if n not in drop_cols and n.endswith('_pmpm_ct')]
nonbin_cms = [n for n in cms_col if n not in drop_cols and (n.endswith('_amt') or n.endswith('eligible_ind'))]
nonbin_hlth = ['hlth_pgm_slvrsnkr_pct_par']
nonbin_med = [n for n in med_col if n not in drop_cols and n.endswith('_days_pmpm')]
nonbin_rx = [n for n in rx_col if n not in drop_cols and (n.endswith('_pmpm_ct') 
                                                          and not n.startswith('rx_overall') and not n.startswith('rx_mail'))]
nonbin_submcc = [n for n in submcc_col if n not in drop_cols and n.endswith('_pmpm_ct')]
nonbin_total = [n for n in total_col if n not in drop_cols and n.endswith('_days_pmpm')]

nonbin_cols = nonbin_betos + nonbin_cms + nonbin_hlth + nonbin_med + nonbin_rx + nonbin_submcc + nonbin_total
# drop columns
encoded_data.drop(nonbin_cols, axis = 1, inplace = True, errors = 'ignore')
encoded_test_data.drop(nonbin_cols, axis = 1, inplace = True, errors = 'ignore')

In [240]:
# drop entire med_col because its encapsulated in total_col
encoded_data.drop([n for n in encoded_data.columns if n.startswith('med_')], axis = 1, inplace = True, errors = 'ignore')
encoded_test_data.drop([n for n in encoded_test_data.columns if n.startswith('med_')], axis = 1, inplace = True, errors = 'ignore')

In [241]:
# split data
from sklearn.model_selection import train_test_split

train_data, val_data, train_y, val_y = train_test_split(encoded_data, y_data, test_size = 0.2, random_state = 44)

In [242]:
# define a function that calculates conditional ratio of response variable
def column_div(df):
    if df[1] != 0:
        return df[0]/df[1]
    else:
        return False

In [243]:
rest_betos = [name for name in encoded_data.columns if name.startswith('betos_')]
rest_bh = [name for name in encoded_data.columns if name.startswith('bh_')]
rest_ccsp = [name for name in encoded_data.columns if name.startswith('ccsp_')]
rest_cms = [name for name in encoded_data.columns if name.startswith('cms_')]
rest_cmsd2 = [name for name in encoded_data.columns if name.startswith('cmsd2_')]
rest_hedis = [name for name in encoded_data.columns if name.startswith('hedis_')]
rest_hlth = [name for name in encoded_data.columns if name.startswith('hlth_')]
rest_lab = [name for name in encoded_data.columns if name.startswith('lab_')]
rest_lang = [name for name in encoded_data.columns if name.startswith('lang_')]
rest_pdc = [name for name in encoded_data.columns if name.startswith('pdc_')]
rest_phy = [name for name in encoded_data.columns if name.startswith('phy_')]
rest_prov = [name for name in encoded_data.columns if name.startswith('prov_')]
rest_rev = [name for name in encoded_data.columns if name.startswith('rev_')]
rest_rx = [name for name in encoded_data.columns if name.startswith('rx_')]
rest_sex = [name for name in encoded_data.columns if name.startswith('sex_')]
rest_smoker = [name for name in encoded_data.columns if name.startswith('smoker_')]
rest_submcc = [name for name in encoded_data.columns if name.startswith('submcc_')]
rest_total = [name for name in encoded_data.columns if name.startswith('total_')]

In [246]:
# keep the one where difference is greater than 2.5
temp_dict = {}
for col in rest_betos:
    temp_dict[col] = pd.crosstab(train_data[col], train_y, margins = False).apply(column_div, axis = 1).to_dict()
temp_dict

{'betos_o1e_ind': {0: 6.694451175187788, 1: 4.910484668644906},
 'betos_t2a_ind': {0: 6.434704830053668, 1: 5.058123817247905},
 'betos_t1a_ind': {0: 4.526752767527675, 1: 6.275528902215559},
 'betos_m2c_ind': {0: 5.938983488132095, 1: 3.4534606205250595},
 'betos_m5b_ind': {0: 6.178183816458304, 1: 2.819506726457399},
 'betos_t1b_ind': {0: 5.80153452685422, 1: 5.814671814671815},
 'betos_m5d_ind': {0: 5.2500580990006975, 1: 6.436142709410548},
 'betos_o1a_ind': {0: 7.155857385398981, 1: 2.340201665935993},
 'betos_m5c_ind': {0: 5.138053097345133, 1: 6.921257290991575},
 'betos_d1c_ind': {0: 5.963016393442623, 1: 3.695970695970696},
 'betos_t1h_ind': {0: 4.848729792147806, 1: 5.925667351129364},
 'betos_o1b_ind': {0: 5.7312593703148424, 1: 9.658682634730539},
 'betos_d1d_ind': {0: 5.905001889882827, 1: 2.641025641025641},
 'betos_t1e_ind': {0: 6.174177290246072, 1: 4.094736842105263},
 'betos_o1g_ind': {0: 5.523106659638025, 1: 6.473387096774194},
 'betos_y2_ind': {0: 6.118729552889858

In [248]:
# keep the one where difference is greter than 10%
for col in credit_ALL_col:
    print(train_data.groupby(train_y)[[col]].apply(lambda x: x.median()).to_dict())

{'credit_bal_1stmtg_30to59dpd': {0: 725.5758085528862, 1: 720.9000494536758}}
{'credit_bal_1stmtg_60to89dpd': {0: 243.81528811520337, 1: 252.49562659056318}}
{'credit_bal_1stmtg_collections': {0: 323.12992203825115, 1: 346.7096547448065}}
{'credit_bal_1stmtg_severederog': {0: 33.04600449609558, 1: 34.398451647553834}}
{'credit_bal_1stmtgcredit_60dpd': {0: 1030.9393711501025, 1: 1099.9973621887152}}
{'credit_bal_agency1stmorg_collectio': {0: 121.60165328570466, 1: 124.81486114598287}}
{'credit_bal_agencyfirstmtg_60dpd': {0: 366.2852279180137, 1: 381.5469126209558}}
{'credit_bal_agencyfirstmtg_new': {0: 5806.567716196336, 1: 5189.736019409852}}
{'credit_bal_autobank': {0: 4746.496072630631, 1: 4558.747028382094}}
{'credit_bal_autobank_new': {0: 2053.3467945214434, 1: 1952.1427525001693}}
{'credit_bal_autofinance': {0: 5067.283022345308, 1: 5379.588131425242}}
{'credit_bal_autofinance_new': {0: 2102.6161351443448, 1: 2207.253898072902}}
{'credit_bal_bankcard_severederog': {0: 373.47458585

- betos: use 'betos_m2c_ind', 'betos_m5b_ind', 'betos_o1a_ind', 'betos_o1b_ind', 'betos_d1d_ind', 'betos_y2_ind'
- bh: use all 
- cci: use cci_score
- ccsp: use 'ccsp_060_ind', 'ccsp_060_ind', 'ccsp_163_ind', 'ccsp_205_ind', 'ccsp_236_ind', 'ccsp_239_ind'
- cms: use all, and cms_ma_risk_score_nbr + cms_rx_risk_score_nbr
- cmsd2: use 'cmsd2_men_mad_ind', 'cmsd2_men_men_substance_ind', 'cmsd2_skn_radiation_ind', 'cmsd2_sns_general_ind'
- cons: use all
- credit: use 'credit_bal_agencyfirstmtg_new', 'credit_bal_heloc_60dpd', 'credit_bal_mtg_bankruptcy', 'credit_bal_mtgcredit_new', 'credit_bal_nonagn1stmorg_bankruptc', 'credit_bal_nonagn1stmorg_collectio', 'credit_bal_nonmtgcredit_60dpd', 'credit_hh_1stmtgcredit', 'credit_hh_1stmtgcredit_new','credit_hh_agencyfirstmtg_new', 'credit_hh_bankcard_severederog', 'credit_hh_totalallcredit_collections', 'credit_num_1stmtgcredit_new', 'credit_num_agencyfirstmtg_new', 'credit_num_mtg_collections', 'credit_num_nonagn1stmorg_bankruptc' 
- dcsi: use dcsi_score
- age: use est_age
- fci: use fci_score
- hcc: use hcc_weighted_sum
- hlth: use 'hlth_pgm_slvrsnkr_par_status'
- mabh: use mabh_seg
- pdc: use 'pdc_ast', 'pdc_dep'
- phy: use 'phy_em_pe_ind', 'phy_em_pi_ind'
- prov: 'prov_spec_ambulance_ind', 'prov_spec_bh_psychiatric_ind', 'prov_spec_chiropractic_ind', 'prov_spec_home_health_ind', 'prov_spec_pain_mgmt_ind'
- rev: use 'rev_cms_ambul_ind', 'rev_cms_er_ind', 'rev_cms_icu_ind', 'rev_cms_phar_ind'
- rucc: use rucc_category
- rx: rx_mail_ind, overall_pmpm_ct, 'rx_gpi2_18_ind', 'rx_gpi2_78_ind', 'rx_gpi2_17_ind', 'rx_gpi2_62_ind', 'rx_bh_ind', 'rx_gpi2_93_ind', 'rx_gpi2_51_ind', 'rx_gpi2_43_ind', 'rx_gpi2_25_ind', 'rx_gpi2_59_ind', 'rx_gpi2_65_ind', 'rx_gpi2_75_ind', 'rx_gpi2_61_ind', 'rx_gpi2_58_ind', 'rx_gpi2_72_ind', 'rx_gpi2_67_ind', 'rx_gpi2_52_ind', 'rx_gpi2_57_ind', 'rx_gpi2_09_ind', 'rx_gpi2_15_ind', 
- smoker: use 'smoker_current_ind'
- submcc: use 'submcc_ano_cns_ind', 'submcc_trm_spnj_ind', 'submcc_inf_sep_ind', 'submcc_res_copd_ind', 'submcc_brn_othr_ind', 'submcc_trm_brn_ind','submcc_hiv_othr_ind', 'submcc_ner_infl_ind', 'submcc_men_alco_ind', 'submcc_sns_dth_ind', 'submcc_men_othr_ind', 'submcc_cer_seq_ind', 'submcc_men_abus_ind', 'submcc_ner_epil_ind', 'submcc_can_skn_ind', 'submcc_can_othr_ind', 'submcc_trm_prly_ind', 'submcc_rar_lup_ind', 'submcc_ano_gu_ind', 'submcc_trm_skul_ind', 'submcc_res_fail_ind', 'submcc_men_schz_ind', 'submcc_sns_othr_ind', 'submcc_can_h/o_ind', 'submcc_trm_f/n_ind', 'submcc_pre_l/d_ind', 'submcc_pre_care_ind', 'submcc_mus_inf_ind', 'submcc_men_depr_ind', 'submcc_cer_hem_ind', 'submcc_rar_hem_ind', 'submcc_rar_scl_ind', 'submcc_rsk_synx_ind'
- total: use 'total_physician_office_visit_ct_pmpm'
- zip: use all zip

In [249]:
f_betos = ['betos_m2c_ind', 'betos_m5b_ind', 'betos_o1a_ind', 'betos_o1b_ind', 'betos_d1d_ind', 'betos_y2_ind']
f_bh = rest_bh
f_cci = ['cci_score']
f_ccsp = ['ccsp_060_ind', 'ccsp_163_ind', 'ccsp_205_ind', 'ccsp_236_ind', 'ccsp_239_ind']
f_cms = rest_cms
f_cmsd2 = ['cmsd2_men_mad_ind', 'cmsd2_men_men_substance_ind', 'cmsd2_skn_radiation_ind', 'cmsd2_sns_general_ind']
f_cons = cons_col
f_credit = ['credit_bal_agencyfirstmtg_new', 'credit_bal_heloc_60dpd', 'credit_bal_mtg_bankruptcy', 'credit_bal_mtgcredit_new', 
            'credit_bal_nonagn1stmorg_bankruptc', 'credit_bal_nonagn1stmorg_collectio', 'credit_bal_nonmtgcredit_60dpd', 
            'credit_hh_1stmtgcredit', 'credit_hh_1stmtgcredit_new','credit_hh_agencyfirstmtg_new', 'credit_hh_bankcard_severederog', 
            'credit_hh_totalallcredit_collections', 'credit_num_1stmtgcredit_new', 'credit_num_agencyfirstmtg_new', 
            'credit_num_mtg_collections', 'credit_num_nonagn1stmorg_bankruptc']
f_dcsi = ['dcsi_score']
f_age = ['est_age']
f_fci = ['fci_score']
f_hcc = ['hcc_weighted_sum']
f_hlth = ['hlth_pgm_slvrsnkr_par_status']
f_lang = lang_col
f_mabh = ['mabh_seg']
f_pdc = ['pdc_ast', 'pdc_dep']
f_phy = ['phy_em_pe_ind', 'phy_em_pi_ind']
f_prov = [ 'prov_spec_ambulance_ind', 'prov_spec_bh_psychiatric_ind', 'prov_spec_chiropractic_ind', 'prov_spec_home_health_ind', 
          'prov_spec_pain_mgmt_ind']
f_rev = ['rev_cms_ambul_ind', 'rev_cms_er_ind', 'rev_cms_icu_ind', 'rev_cms_phar_ind']
f_rucc = rucc_col
f_rx = ['rx_gpi2_18_ind', 'rx_gpi2_78_ind', 'rx_gpi2_17_ind', 'rx_gpi2_62_ind', 'rx_bh_ind', 'rx_gpi2_93_ind', 'rx_gpi2_51_ind', 
        'rx_gpi2_43_ind', 'rx_gpi2_25_ind', 'rx_gpi2_59_ind', 'rx_gpi2_65_ind', 'rx_gpi2_75_ind', 'rx_gpi2_61_ind', 'rx_gpi2_58_ind', 
        'rx_gpi2_72_ind', 'rx_gpi2_67_ind', 'rx_gpi2_52_ind', 'rx_gpi2_57_ind', 'rx_gpi2_09_ind', 'rx_gpi2_15_ind']
f_smoker = ['smoker_current_ind']
f_submcc = ['submcc_ano_cns_ind', 'submcc_trm_spnj_ind', 'submcc_inf_sep_ind', 'submcc_res_copd_ind', 'submcc_brn_othr_ind', 
            'submcc_trm_brn_ind','submcc_hiv_othr_ind', 'submcc_ner_infl_ind', 'submcc_men_alco_ind', 'submcc_sns_dth_ind', 
            'submcc_men_othr_ind', 'submcc_cer_seq_ind', 'submcc_men_abus_ind', 'submcc_ner_epil_ind', 'submcc_can_skn_ind', 
            'submcc_can_othr_ind', 'submcc_trm_prly_ind', 'submcc_rar_lup_ind', 'submcc_ano_gu_ind', 'submcc_trm_skul_ind', 
            'submcc_res_fail_ind', 'submcc_men_schz_ind', 'submcc_sns_othr_ind', 'submcc_can_h/o_ind', 'submcc_trm_f/n_ind', 
            'submcc_pre_l/d_ind', 'submcc_pre_care_ind', 'submcc_mus_inf_ind', 'submcc_men_depr_ind', 'submcc_cer_hem_ind', 
            'submcc_rar_hem_ind', 'submcc_rar_scl_ind', 'submcc_rsk_synx_ind']
f_total = ['total_physician_office_visit_ct_pmpm']
f_zip = zip_col

In [250]:
keep_cols = []
keep_cols = f_betos + f_bh + f_cci + f_ccsp + f_cms + f_cmsd2 + f_cons + f_credit + f_dcsi + f_age + f_fci + f_hcc + f_hlth + f_lang
keep_cols = keep_cols + f_mabh + f_pdc + f_phy + f_prov + f_rev + f_rucc + f_rx + f_smoker + f_submcc + f_total + f_zip

In [251]:
train_data = train_data[keep_cols].copy()
val_data = val_data[keep_cols].copy()
encoded_test_data = encoded_test_data[keep_cols].copy()

In [252]:
train_data.shape

(55657, 203)

In [172]:
# export dropped file to csv
train_data.to_csv('dropped_training.csv', index = False)
val_data.to_csv('dropped_val.csv', index = False)
encoded_test_data.to_csv('dropped_holdout.csv', index = False)

In [181]:
train_y.to_csv('train_y.csv', index = False)
val_y.to_csv('val_y.csv', index = False)

In [173]:
# dropped_train_data = pd.read_csv('dropped_training.csv.csv', low_memory = False)
# dropped_val_data = pd.read_csv('dropped_val.csv', low_memory = False)
# dropped_test_data = pd.read_csv('dropped_holdout.csv', low_memory = False)

dropped_train_data = train_data.copy()
dropped_val_data = val_data.copy()
dropped_test_data = encoded_test_data.copy()

In [176]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_train = pd.DataFrame(scaler.fit_transform(dropped_train_data), columns = dropped_train_data.columns)
scaled_val = pd.DataFrame(scaler.transform(dropped_val_data), columns = dropped_val_data.columns)
scaled_test = pd.DataFrame(scaler.transform(dropped_test_data), columns = dropped_test_data.columns)

In [177]:
scaled_train.head()

,betos_m2c_ind,betos_m5b_ind,betos_o1a_ind,betos_o1b_ind,betos_d1d_ind,betos_y2_ind,bh_adtp_ind,bh_aoth_ind,bh_bipr_ind,bh_cdal_ind,...,zip_cd_75,zip_cd_72,zip_cd_88,zip_cd_73,zip_cd_63,zip_cd_44,zip_cd_77,zip_cd_57,zip_cd_31,zip_cd_76
0,-0.186252,-0.255354,2.510981,-0.181764,-0.124684,-0.256430,-0.107171,-0.462658,-0.175987,-0.17265,...,-0.041350,-0.031451,-0.023607,-0.027806,-0.023607,-0.032576,-0.023985,-0.015862,-0.028446,-0.015285
1,5.369069,-0.255354,2.510981,-0.181764,-0.124684,3.899696,-0.107171,2.161426,-0.175987,-0.17265,...,-0.041350,-0.031451,-0.023607,-0.027806,-0.023607,-0.032576,-0.023985,-0.015862,-0.028446,-0.015285
2,-0.186252,-0.255354,-0.398251,-0.181764,-0.124684,-0.256430,-0.107171,-0.462658,-0.175987,-0.17265,...,24.183944,-0.031451,-0.023607,-0.027806,-0.023607,-0.032576,-0.023985,-0.015862,-0.028446,-0.015285
3,-0.186252,-0.255354,-0.398251,-0.181764,-0.124684,3.899696,-0.107171,-0.462658,-0.175987,-0.17265,...,-0.041350,-0.031451,-0.023607,-0.027806,-0.023607,-0.032576,-0.023985,-0.015862,-0.028446,-0.015285
4,-0.186252,-0.255354,-0.398251,-0.181764,-0.124684,-0.256430,-0.107171,-0.462658,-0.175987,-0.17265,...,-0.041350,-0.031451,-0.023607,-0.027806,-0.023607,-0.032576,-0.023985,-0.015862,-0.028446,-0.015285


In [178]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(penalty='l1', solver= 'liblinear', random_state= 44, max_iter= 150)
lr_model.fit(scaled_train, train_y)


from sklearn.model_selection import cross_validate

cv_results_lr = cross_validate(lr_model, X = scaled_train, y = train_y, scoring = 'roc_auc', cv = 10)
cv_results_lr['test_score']

array([0.74255329, 0.73886044, 0.7400352 , 0.73777176, 0.72509216,
       0.72975281, 0.75137096, 0.74500788, 0.72978225, 0.73414587])

In [253]:
cv_results_lr['test_score'].mean()

0.7374372630590298

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state = 44)

cv_results_rf = cross_validate(rf_model, X = scaled_train, y = train_y, scoring = 'roc_auc', cv = 10)
cv_results_rf['test_score']

array([0.72463351, 0.72852276, 0.73086172, 0.72693704, 0.72241093,
       0.71990805, 0.72379987, 0.73777093, 0.7145182 , 0.71292828])

In [179]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 1000, random_state = 44)

cv_results_xgb = cross_validate(xgb_model, X = scaled_train, y = train_y, scoring = 'roc_auc', cv = 10)
cv_results_xgb['test_score']

array([0.73438488, 0.73638333, 0.73039071, 0.72737442, 0.72454795,
       0.72390374, 0.74298371, 0.73759279, 0.72674238, 0.72905193])

In [183]:
cv_results_rf['test_score'].mean()

0.7242291283972989

In [185]:
from sklearn.feature_selection import SelectKBest, f_classif

total_feature_num = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]

for num in total_feature_num:
    selector = SelectKBest(f_classif, k = num)
    temp_X = selector.fit_transform(scaled_train, train_y)
    new_col = scaled_train.columns[selector.get_support()]
    
    new_train = scaled_train[new_col]
    temp_lr_model = LogisticRegression(random_state = num, max_iter = 150)
    temp_cv_results = cross_validate(temp_lr_model, X = new_train, y = train_y, scoring = 'roc_auc', cv = 10)

    print(num, ' - ', temp_cv_results['test_score'].mean())

50  -  0.7338490179791616
60  -  0.7354861993808879
70  -  0.7379715204394868
80  -  0.7378735763519708
90  -  0.7376050981355246
100  -  0.7377763437675945
110  -  0.7387443116568815
120  -  0.7389109079703755
130  -  0.7391143641122839
140  -  0.7389146748659883
150  -  0.7387529433620007


In [213]:
# so total feature num = 130
selector = SelectKBest(f_classif, k = 130)
temp_X = selector.fit_transform(scaled_train, train_y)
new_col = scaled_train.columns[selector.get_support()]

new_train = scaled_train[new_col]
new_val = scaled_val[new_col]
new_test = scaled_test[new_col]

In [188]:
rf_model_2 = RandomForestClassifier(random_state = 30)

cv_results_rf_2 = cross_validate(rf_model_2, X = new_train, y = train_y, scoring = 'roc_auc', cv = 10)
cv_results_rf_2['test_score'].mean()

0.7204055809807228

In [214]:
final_model = LogisticRegression(penalty='l1', solver= 'liblinear', random_state = 130, max_iter = 150)
final_model.fit(new_train, train_y)

from sklearn.metrics import roc_auc_score
roc_auc_score(val_y, final_model.decision_function(new_val))

0.7356437554121261

In [215]:
test_score = final_model.predict_proba(new_test)[:][:,1]

In [216]:
test_score_df = pd.DataFrame(test_score, columns= ['SCORE'])

In [217]:
test_score_df.to_csv('test_score2.csv', index = False)

In [219]:
final_model.predict_proba(new_test)[:]

array([[0.39325698, 0.60674302],
       [0.91077363, 0.08922637],
       [0.81073995, 0.18926005],
       ...,
       [0.84972846, 0.15027154],
       [0.95226246, 0.04773754],
       [0.69341381, 0.30658619]])

In [218]:
test_score

array([0.60674302, 0.08922637, 0.18926005, ..., 0.15027154, 0.04773754,
       0.30658619])

In [211]:
from sklearn.feature_selection import SelectKBest, f_classif

total_feature_num = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]

for num in total_feature_num:
    selector = SelectKBest(f_classif, k = num)
    temp_X = selector.fit_transform(scaled_train, train_y)
    new_col = scaled_train.columns[selector.get_support()]
    
    new_train = scaled_train[new_col]
    temp_lr_model = LogisticRegression(penalty='l1', solver= 'liblinear', random_state = num, max_iter = 150)
    temp_cv_results = cross_validate(temp_lr_model, X = new_train, y = train_y, scoring = 'roc_auc', cv = 10)

    print(num, ' - ', temp_cv_results['test_score'].mean())

50  -  0.7338669563453319
60  -  0.7355287795355574
70  -  0.7380134803527159
80  -  0.7379087055152662
90  -  0.7376503837652224
100  -  0.7378203962821019
110  -  0.7387956768907917
120  -  0.738964724263116
130  -  0.7391612468577107
140  -  0.7389712495349658
150  -  0.7388174035667945
